# Финальная сборка и заливка табличек с данными о потенциальных местах размещения постаматов

Потенциальные места размещения:
- многоквартирный дом
- киоск
- МФЦ
- библиотека
- дом культуры или клуб
- cпортивный объект

Данные, которые будут лежать в табличках:
- Административный округ
- Район
- Тип объекта размещения
- Координата точки размещения
- Адрес точки размещения

Исходные данные скачиваем в формате json из:
- Нестационарные торговые объекты по реализации печатной продукции: https://data.mos.ru/opendata/2781
- Нестационарные торговые объекты: https://data.mos.ru/opendata/619
- Многофункциональные центры предоставления государственных и муниципальных услуг https://data.mos.ru/opendata/-mnogofunktsionalnye-tsentry-predostavleniya-gosudarstvennyh-uslug
- Библиотеки города: https://data.mos.ru/opendata/7702155262-biblioteki
- Дома культуры и клубы: https://data.mos.ru/opendata/7702155262-doma-kultury-i-kluby
- Спортивные объекты города Москвы: https://data.mos.ru/opendata/7708308010-sportivnye-obekty-goroda-moskvy
- данные о жилых домах - из уже залитой в БД <code>apartment_houses_all_data</code>

Заливаются таблички:
- <code>kiosks_all_data</code>
- <code>mfc_all_data</code>
- <code>libs_all_data</code>
- <code>clubs_all_data</code>
- <code>sports_all_data</code>
- <code>all_objects_data</code>

Перед прогоном тетрадки, если надо обновить <code>apartment_houses_all_data</code>, нужно запустить подготовку данных:
- <code>step1_prepare_dmr_houses_data.ipynb</code>
- <code>step2_prepare_gis_houses_data.ipynb</code>
- <code>step3_apartment_houses_all_data.ipynb</code>

In [1]:
%load_ext autoreload
import os
from typing import List, Dict, Optional
from warnings import warn
import json
import pandas as pd
import numpy as np

from postamats.utils import prepare_data
from postamats.utils.connections import DB
from postamats.global_constants import INFRA_GEODATA_COL, INFRA_NEEDED_COLS_BY_OBJECTS, INFRA_TABLES_NAMES_BY_OBJECTS,\
    MANDATORY_COLS, OBJECT_ID_COL, OBJECT_TYPE_COL
from postamats.utils.connections import PATH_TO_ROOT

pd.set_option('display.max_columns', None)

In [2]:
CONFIG_PATH = os.path.join(PATH_TO_ROOT, 'db_config.json')
with open(CONFIG_PATH, mode='r') as db_file:
    db_config = json.load(db_file)

database = DB(db_config)

In [3]:
KIOSKS_FOLDER = os.path.join(PATH_TO_ROOT, 'data', 'kiosks')
# Нестационарные торговые объекты
KIOSKS_FILE_1 = 'data-104108-2022-10-23.json'
# Нестационарные торговые объекты по реализации печатной продукции
KIOSKS_FILE_2 = 'data-101756-2022-10-25.json'
# Многофункциональные центры предоставления государственных и муниципальных услуг 
MFC_FOLDER = os.path.join(PATH_TO_ROOT, 'data', 'mfc')
MFC_FILE = 'data-2728-2022-10-26.json'
# Библиотеки
LIBS_FOLDER = os.path.join(PATH_TO_ROOT, 'data', 'libs')
LIBS_FILE = 'data-7361-2022-10-19.json'
# дома культуры и клубы
CLUBS_FOLDER = os.path.join(PATH_TO_ROOT, 'data', 'clubs')
CLUBS_FILE = 'data-7392-2022-10-27.json'
# спортивные объекты
SPORTS_FOLDER = os.path.join(PATH_TO_ROOT, 'data', 'sports')
SPORTS_FILE = 'data-8303-2022-08-08.json'

In [4]:
kiosks1 = pd.read_json(os.path.join(KIOSKS_FOLDER, KIOSKS_FILE_1), encoding='cp1251').dropna(axis=1, how='all')
kiosks2 = pd.read_json(os.path.join(KIOSKS_FOLDER, KIOSKS_FILE_2), encoding='cp1251').dropna(axis=1, how='all')
kiosks = pd.concat([kiosks1, kiosks2])

mfc = pd.read_json(os.path.join(MFC_FOLDER, MFC_FILE), encoding='cp1251').dropna(axis=1, how='all')

libs = pd.read_json(os.path.join(LIBS_FOLDER, LIBS_FILE), encoding='cp1251').dropna(axis=1, how='all')

clubs = pd.read_json(os.path.join(CLUBS_FOLDER, CLUBS_FILE), encoding='cp1251').dropna(axis=1, how='all')

sports = pd.read_json(os.path.join(SPORTS_FOLDER, SPORTS_FILE), encoding='cp1251').dropna(axis=1, how='all')

In [5]:
objects_data_map = {
    'киоск': kiosks1,
    'МФЦ': mfc,
    'библиотека': libs,
    'дом культуры или клуб': clubs,
    'cпортивный объект': sports
    }
prepared_objects_list = []
for obj_type, obj_data in objects_data_map.items():
    print(obj_type)
    load_to_bd_data = prepare_data.prepare_infrastructure_objects(obj_data,
                                                                  INFRA_GEODATA_COL,
                                                                  obj_type,
                                                                  needed_cols=INFRA_NEEDED_COLS_BY_OBJECTS[obj_type])
    prepared_objects_list.append(load_to_bd_data.copy())
    database.load_to_bd(load_to_bd_data, INFRA_TABLES_NAMES_BY_OBJECTS[obj_type])

киоск
prepare_infrastructure_objects started ... 


  0%|          | 0/2515 [00:00<?, ?it/s]/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/helpers.py:85: UserWarning: coords is dot
  warn('coords is dot')
100%|██████████| 2515/2515 [00:00<00:00, 115664.02it/s]

prepare_infrastructure_objects finished


МФЦ
prepare_infrastructure_objects started ... 


  0%|          | 0/139 [00:00<?, ?it/s]/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/helpers.py:85: UserWarning: coords is dot
  warn('coords is dot')
100%|██████████| 139/139 [00:00<00:00, 73086.15it/s]

...

  0%|          | 0/76 [00:00<?, ?it/s]/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/helpers.py:85: UserWarning: coords is dot
  warn('coords is dot')
/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/helpers.py:88: UserWarning: coords is line
  warn('coords is line')
100%|██████████| 76/76 [00:00<00:00, 66313.11it/s]

prepare_infrastructure_objects finished


cпортивный объект
prepare_infrastructure_objects started ... 


  0%|          | 0/346 [00:00<?, ?it/s]/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/helpers.py:85: UserWarning: coords is dot
  warn('coords is dot')
100%|██████████| 346/346 [00:00<00:00, 94101.23it/s]

prepare_infrastructure_objects finished


In [6]:
apartment_houses = database.get_table_from_bd('apartment_houses_all_data')

Connection to PostgreSQL DB successful


/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:88: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [7]:
all_objects_data = pd.concat([apartment_houses] + prepared_objects_list, ignore_index=True)
all_objects_data = all_objects_data[MANDATORY_COLS].drop_duplicates(subset=[OBJECT_ID_COL])

In [9]:
all_objects_data

,object_id,adm_area,district,object_type,lat,lon,address
0,c7ad8391b30a404697edcfdea4ba10fa7ea552b4e09269...,Северо-Западный административный округ,муниципальный округ Щукино,многоквартирный дом,55.807827,37.453977,"город Москва, Авиационная улица, дом 65, корпу..."
1,7b9adf96d39c350de4fdf992f1ff2e497b1f8f5838cd9f...,Южный административный округ,муниципальный округ Зябликово,многоквартирный дом,55.623560,37.739392,"Российская Федерация, город Москва, внутригоро..."
2,eea53cceda9ab91905922e4fad25b4b146cfb647488f23...,Северо-Восточный административный округ,муниципальный округ Алтуфьевский,многоквартирный дом,55.885641,37.593456,"Российская Федерация, город Москва, внутригоро..."
3,9d0f9c193a2b3c86da7b807d19c8884113b0d2421b7155...,Южный административный округ,муниципальный округ Чертаново Северное,многоквартирный дом,55.630294,37.617104,"Российская Федерация, город Москва, внутригоро..."
4,e6c1cae0d69e48fc18621c51ffd1fd2b54dbf13c6d38b5...,Восточный административный округ,муниципальный округ Новогиреево,многоквартирный дом,55.742228,37.784629,"город Москва, Кусковская улица, дом 29, корпус..."
...,...,...,...,...,...,...,...
41367,0bd2902029c3d36f6ead25fe181b92b25bd816cdad8a5b...,Зеленоградский административный округ,район Крюково,cпортивный объект,55.974472,37.155215,"Российская Федерация, город Москва, внутригоро..."
41368,c350a792c7f85c3a19cceccabbee229e762627670c3d2f...,Зеленоградский административный округ,район Крюково,cпортивный объект,55.962885,37.174221,"город Москва, город Зеленоград, проектируемый ..."
41369,703ecf2861165481f798ca73c3b17a3f3d5eabbdbc7b94...,Южный административный округ,район Орехово-Борисово Северное,cпортивный объект,55.614951,37.713617,"Российская Федерация, город Москва, внутригоро..."
41370,9a4d692b68af938eeb2710c530c946516dcb39d85a9e62...,Западный административный округ,район Фили-Давыдково,cпортивный объект,55.734761,37.456232,"Российская Федерация, город Москва, внутригоро..."


In [10]:
assert all_objects_data[OBJECT_ID_COL].duplicated().sum() == 0, f'Дубли: {all_objects_data[OBJECT_ID_COL].duplicated().sum()}'
assert all_objects_data[OBJECT_TYPE_COL].unique().shape[0] == 6, f'Типы: {all_objects_data[OBJECT_TYPE_COL].unique().shape[0]}'

In [11]:
database.load_to_bd(all_objects_data, 'all_objects_data')
database.get_table_from_bd('all_objects_data')

Connection to PostgreSQL DB successful


/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:88: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,object_id,adm_area,district,object_type,lat,lon,address
0,c7ad8391b30a404697edcfdea4ba10fa7ea552b4e09269...,Северо-Западный административный округ,муниципальный округ Щукино,многоквартирный дом,55.807827,37.453977,"город Москва, Авиационная улица, дом 65, корпу..."
1,7b9adf96d39c350de4fdf992f1ff2e497b1f8f5838cd9f...,Южный административный округ,муниципальный округ Зябликово,многоквартирный дом,55.623560,37.739392,"Российская Федерация, город Москва, внутригоро..."
2,eea53cceda9ab91905922e4fad25b4b146cfb647488f23...,Северо-Восточный административный округ,муниципальный округ Алтуфьевский,многоквартирный дом,55.885641,37.593456,"Российская Федерация, город Москва, внутригоро..."
3,9d0f9c193a2b3c86da7b807d19c8884113b0d2421b7155...,Южный административный округ,муниципальный округ Чертаново Северное,многоквартирный дом,55.630294,37.617104,"Российская Федерация, город Москва, внутригоро..."
4,e6c1cae0d69e48fc18621c51ffd1fd2b54dbf13c6d38b5...,Восточный административный округ,муниципальный округ Новогиреево,многоквартирный дом,55.742228,37.784629,"город Москва, Кусковская улица, дом 29, корпус..."
...,...,...,...,...,...,...,...
41367,0bd2902029c3d36f6ead25fe181b92b25bd816cdad8a5b...,Зеленоградский административный округ,район Крюково,cпортивный объект,55.974472,37.155215,"Российская Федерация, город Москва, внутригоро..."
41368,c350a792c7f85c3a19cceccabbee229e762627670c3d2f...,Зеленоградский административный округ,район Крюково,cпортивный объект,55.962885,37.174221,"город Москва, город Зеленоград, проектируемый ..."
41369,703ecf2861165481f798ca73c3b17a3f3d5eabbdbc7b94...,Южный административный округ,район Орехово-Борисово Северное,cпортивный объект,55.614951,37.713617,"Российская Федерация, город Москва, внутригоро..."
41370,9a4d692b68af938eeb2710c530c946516dcb39d85a9e62...,Западный административный округ,район Фили-Давыдково,cпортивный объект,55.734761,37.456232,"Российская Федерация, город Москва, внутригоро..."
